# Iris 분류
## VSCODE에서는 테라스 Import에 대한 이슈가 있는듯. 
##Jupyter notebook(가상환경)에서 우선적으로 학습 하는것을 추천.

In [25]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [26]:
from sklearn.datasets import load_iris
iris = load_iris()

In [27]:
# data, target, target_names, feature_names, DESCR
data = iris.data
label = iris.target
columns = iris.feature_names
df = pd.DataFrame(data, columns=columns)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = \
    train_test_split(data, label, test_size=0.2, 
                     shuffle=True, stratify=label, random_state=2019)

### Logistic Regression

- 모델 객체 생성

In [29]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(verbose=1)

- 모델 학습

In [30]:
lr.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

- 학습한 결과를 저장

In [31]:
import joblib
joblib.dump(lr, 'model/iris_lr.pkl')

['model/iris_lr.pkl']

### Support Vector Machine

In [32]:
from sklearn.svm import SVC
svc = SVC()

In [33]:
svc.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [34]:
joblib.dump(svc, 'model/iris_svm.pkl')

['model/iris_svm.pkl']

### Decision Tree

In [35]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [36]:
dtc.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [37]:
joblib.dump(dtc, 'model/iris_dt.pkl')

['model/iris_dt.pkl']

### 저장된 모델로 정확도 산출

In [38]:
model_lr = joblib.load('model/iris_lr.pkl')
model_svm = joblib.load('model/iris_svm.pkl')
model_dt = joblib.load('model/iris_dt.pkl')

In [39]:
y_pred_lr = model_lr.predict(x_test)
y_pred_svm = model_svm.predict(x_test)
y_pred_dt = model_dt.predict(x_test)

In [40]:
from sklearn.metrics import accuracy_score
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(acc_lr, acc_svm, acc_dt)

0.9666666666666667 0.9666666666666667 0.9666666666666667


### 한개의 테스트 값 예측

In [41]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [42]:
test_data = np.array([6.7, 3.0, 5.2, 2.3]).reshape(1, 4)

In [43]:
index_lr = model_lr.predict(test_data)[0]
index_svm = model_svm.predict(test_data)[0]
index_dt = model_dt.predict(test_data)[0]

In [44]:
sp_names = ['Setosa', 'Versicolor', 'Virginica']
print(sp_names[index_lr], sp_names[index_svm], sp_names[index_dt])

Virginica Virginica Virginica


## Deep Learning Model

In [73]:
# 딥러닝 모델 설정

from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

model = Sequential([
    Dense(12, input_shape=(4,), activation='relu'),
    Dense(8, input_shape=(4,), activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 12)                60        
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 컴파일 
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [64]:
from tensorflow.python.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="model/iris_deep.hdf5", 
                               monitor='val_loss', 
                               verbose=1, save_best_only=True)

In [65]:
# One hot encoding
Y_encoded = keras.utils.to_categorical(y_train)
Y_encoded[0], Y_encoded[50], Y_encoded[100]

(array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32))

In [74]:
# 모델 학습
tensorflow.python.keras.model.fit(x_train, Y_encoded, epochs=100, batch_size=30, verbose=2,
          validation_split=0.2, callbacks=[checkpointer])

NameError: name 'tensorflow' is not defined

In [70]:
from keras.models import load_model
model_deep = load_model("model/iris_deep.hdf5")

ModuleNotFoundError: No module named 'keras'

In [41]:
model.evaluate(x_test, keras.utils.to_categorical(y_test))[1]

30/30 [==============================] - 0s 33us/step


0.9333333373069763

In [47]:
model_deep.predict_classes(test_data)[0]

2

In [45]:
model_deep.predict(test_data)

array([[0.0120978 , 0.3980688 , 0.58983344]], dtype=float32)

In [44]:
np.argmax(model_deep.predict(test_data)[0])

2